<a href="https://colab.research.google.com/github/doantronghieu/DEEP-LEARNING/blob/main/CSR_TF_Dev_Pro_Cert/C1/C1_W4_Lab_2_image_generator_with_validation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<a href="https://colab.research.google.com/github/https-deeplearning-ai/tensorflow-1-public/blob/master/C1/W4/ungraded_labs/C1_W4_Lab_2_image_generator_with_validation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Ungraded Lab: ImageDataGenerator with a Validation Set

In this lab, you will continue using the `ImageDataGenerator` class to prepare the `Horses or Humans` dataset. This time, you will add a validation set so you can also measure how well the model performs on data it hasn't seen.

**IMPORTANT NOTE:** This notebook is designed to run as a Colab. Running it on your local machine might result in some of the code blocks throwing errors.

Run the code blocks below to download the datasets `horse-or-human.zip` and `validation-horse-or-human.zip` respectively.

In [ ]:
# Download the training set
!wget https://storage.googleapis.com/tensorflow-1-public/course2/week3/horse-or-human.zip
# Download the validation set
!wget https://storage.googleapis.com/tensorflow-1-public/course2/week3/validation-horse-or-human.zip

Then unzip both archives.

In [ ]:
import zipfile

# Unzip training set
local_zip = 'horse-or-human.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('./horse-or-human')


# Unzip validation set
local_zip = './validation-horse-or-human.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('./validation-horse-or-human')

zip_ref.close()

Similar to the previous lab, you will define the directories containing your images. This time, you will include those with validation data.

In [ ]:
import os

# Directory with the training horse pictures
train_horse_dir = os.path.join('./horse-or-human/horses')

# Directory with the training human pictures
train_human_dir = os.path.join('./horse-or-human/humans')

# Directory with the validation horse pictures
validation_horse_dir = os.path.join('./validation-horse-or-human/horses')

# Directory with the validation human pictures
validation_human_dir = os.path.join('./validation-horse-or-human/humans')

print(train_horse_dir)
print(train_human_dir)
print(validation_horse_dir)
print(validation_human_dir)

Now see what the filenames look like in these directories:

In [ ]:
train_horse_names = os.listdir(train_horse_dir)
train_human_names = os.listdir(train_human_dir)
validation_horse_names = os.listdir(validation_horse_dir)
validation_human_names = os.listdir(validation_human_dir)

print(f'TRAIN SET HORSES: {train_horse_names[:10]}\n')
print(f'TRAIN SET HUMANS: {train_human_names[:10]}\n')
print(f'VAL SET HORSES: {validation_horse_names[:10]}\n')
print(f'VAL SET HUMANS: {validation_human_names[:10]}\n')

You can find out the total number of horse and human images in the directories:

In [ ]:
print(f'Total training horse images:   {len(os.listdir(train_horse_dir))}')
print(f'Total training human images:   {len(os.listdir(train_human_dir))}')
print(f'Total validation horse images: {len(os.listdir(validation_horse_dir))}')
print(f'Total validation human images: {len(os.listdir(validation_human_dir))}')

Now take a look at a few pictures to get a better sense of what they look like. First, configure the `matplotlib` parameters:

In [ ]:
%matplotlib inline

import matplotlib.pyplot as plt
import matplotlib.image as mpimg

# Parameters for our graph; we'll output images in a 4x4 configuration
nrows = 4
ncols = 4

# Index for iterating over images
pic_index = 0

Now, display a batch of 8 horse and 8 human pictures. You can rerun the cell to see a fresh batch each time:

In [ ]:
# Set up matplotlib fig, and size it to fit 4x4 pics
fig = plt.gcf()
fig.set_size_inches(ncols * 4, nrows * 4)

pic_index += 8
next_horse_pix = [os.path.join(train_horse_dir, fname)
                    for fname in train_horse_names[pic_index-8:pic_index]]
next_human_pix = [os.path.join(train_human_dir, fname)
                    for fname in train_human_names[pic_index-8:pic_index]]

print(len(next_horse_pix), '-', next_horse_pix)
print(len(next_human_pix), '-', next_human_pix)

for i, img_path in enumerate(next_horse_pix + next_human_pix):
    # Set up subplot; subplot indices start at 1
    sp = plt.subplot(nrows, ncols, i + 1)
    sp.axis('Off') # No show axes

    img = mpimg.imread(img_path)
    plt.imshow(img)

plt.show()

## Building a Small Model from Scratch

You will define the same model architecture as before:

In [ ]:
import tensorflow as tf
import tensorflow.keras as tfk
from tensorflow.keras import layers, optimizers, models, losses
from tensorflow import nn

model = models.Sequential([
    # Input shape: Desired size of the image 300x300 with 3 bytes color

    # First convolution
    layers.Conv2D(16, (3, 3), activation=nn.relu, input_shape=(300, 300, 3)),
    layers.MaxPooling2D(2, 2),

    # Second convolution
    layers.Conv2D(32, (3, 3), activation=nn.relu),
    layers.MaxPooling2D(2, 2),

    # Third convolution
    layers.Conv2D(64, (3, 3), activation=nn.relu),
    layers.MaxPooling2D(2, 2),

    # Fourth convolution
    layers.Conv2D(64, (3, 3), activation=nn.relu),
    layers.MaxPooling2D(2, 2),

    # Fifth convolution
    layers.Conv2D(64, (3, 3), activation=nn.relu),
    layers.MaxPooling2D(2, 2),       

    # Flatten the results to feed into a DNN
    layers.Flatten(),
    layers.Dense(512, activation=nn.relu),  # 512 neuron hidden layer
    # Only 1 output neuron
    # It will contains a value from 0-1 where 0 for 1 class ('horses') and 1
    #  for the other ('humans')
    layers.Dense(1, activation=nn.sigmoid)                    
])

You can review the network architecture and the output shapes with `model.summary()`.

In [ ]:
model.summary()

You will also use the same compile settings as before:

In [ ]:
model.compile(loss=losses.binary_crossentropy,
              optimizer=optimizers.RMSprop(learning_rate=0.001),
              metrics=['accuracy'])

### Data Preprocessing

Now you will setup the data generators. It will mostly be the same as last time but notice the additional code to also prepare the validation data. It will need to be instantiated separately and also scaled to have `[0,1]` range of pixel values.

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

BATCH_SIZE = 128

# All images will be rescaled by 1. / 255
train_datagen      = ImageDataGenerator(rescale=1/255)
validation_datagen = ImageDataGenerator(rescale=1/255)

# Flow training images in batches of 128 using train_datagen generator
train_generator = train_datagen.flow_from_directory(
    './horse-or-human/', # The source directory for training images
    target_size=(300, 300), # All images will be resized to 300x300
    batch_size=BATCH_SIZE,
    class_mode='binary') # Since we use binary_crossentropy loss, we need binary labels

# Flow validation images in batches of 128 using validation_datagen generator
validation_generator = validation_datagen.flow_from_directory(
    './validation-horse-or-human/', # The source directory for validation images
    target_size=(300, 300), # All images will be resized to 300x300
    batch_size=32,
    class_mode='binary')

print(len(train_generator), '-', len(validation_generator))

### Training
Now train the model for 15 epochs. Here, you will pass parameters for `validation_data` and `validation_steps`. With these, you will notice additional outputs in the print statements: `val_loss` and `val_accuracy`. Notice that as you train with more epochs, your training accuracy might go up but your validation accuracy goes down. This can be a sign of overfitting and you need to prevent your model from reaching this point.

In [ ]:
history = model.fit(train_generator, steps_per_epoch=8, epochs=15, verbose=1,
                    validation_data=validation_generator, validation_steps=8)

### Model Prediction

Now take a look at actually running a prediction using the model. This code will allow you to choose 1 or more files from your file system, upload them, and run them through the model, giving an indication of whether the object is a horse or a human.

**Important Note:** Due to some compatibility issues, the following code block will result in an error after you select the images(s) to upload if you are running this notebook as a `Colab` on the `Safari` browser. For all other browsers, continue with the next code block and ignore the next one after it.

In [ ]:
import numpy as np
from google.colab import files
from keras.preprocessing import image

uploaded = files.upload()

for fn in uploaded.keys():

    # Predicting images
    path = '/content/' + fn

    img = image.load_img(path, target_size=(300, 300))
    img = image.img_to_array(img)
    img = img / 255
    img = np.expand_dims(img, axis=0)

    images = np.vstack([img])
    classes = model.predict(images, batch_size=10)
    
    if (classes[0] > 0.5):
        print(f'There is a human in image_{i+1}')
    else:
        print(f'There is a horse in image_{i+1}')

### Visualizing Intermediate Representations

As before, you can plot how the features are transformed as it goes through each layer.

In [ ]:
import numpy as np
import random
from tensorflow.keras.preprocessing.image import img_to_array, load_img

#   Define a new model that will take an image as input, and will output
# intermediate representations for all layers in the previous model after
# the first.
successive_outputs = [layer.output for layer in model.layers[1:]]
visualization_model = models.Model(inputs=model.input,
                                   outputs=successive_outputs)

# Prepare a random input image from the training set.
horse_img_files = [os.path.join(train_horse_dir, fn) for fn in train_horse_names]
human_img_files = [os.path.join(train_human_dir, fn) for fn in train_human_names]
img_path = random.choice(horse_img_files + human_img_files)

img = load_img(img_path, target_size=(300, 300)) # This is a PIL image
x = img_to_array(img)         # Numpy array with shape (300, 300, 3)
x = x.reshape((1,) + x.shape) # Numpy array with shape (1, 300, 300, 3)
x = x / 255 # Scale by 1 / 255

#   Run the image through the network, thus obtaining all intermediate
# representations for this image
successive_feature_maps = visualization_model.predict(x)

# These are the names of the layers, so we can have them as part of the plot
layer_names = [layer.name for layer in model.layers[1:]]

# Display the representations
for layer_name, feature_map in zip(layer_names, successive_feature_maps):
    # Just do this for the conv/maxpool layers, not the fully-connected layers
    if (len(feature_map.shape) == 4):

        # Number of features in feature map
        # The feature map has shape (1, size, size, n_features)
        n_features = feature_map.shape[-1] 
        size       = feature_map.shape[1]

        # Tile the images in this matrix
        display_grid = np.zeros((size, size * n_features))
        for i in range(n_features):
            x = feature_map[0, :, :, i]
            x -= x.mean()
            x /= x.std()
            x *= 64
            x += 128
            x = np.clip(x, 0, 255).astype('uint8')

            # Tile each filter into this big horizontal grid
            display_grid[:, i * size : (i + 1) * size] = x

        # Display the grid
        scale = 20. / n_features
        plt.figure(figsize=(scale * n_features, scale))
        plt.title(layer_name)
        plt.grid(False)
        plt.imshow(display_grid, aspect='auto', cmap='viridis');

## Clean Up

Before running the next exercise, run the following cell to terminate the kernel and free memory resources:

In [ ]:
# import os, signal
# os.kill(os.getpid(), signal.SIGKILL)